### portmy database: stocks table
### stock database: buy, price tables

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
week_str = today.strftime('%Y-%U')
today, week_str

(datetime.date(2022, 1, 3), '2022-01')

In [2]:
today = today - timedelta(days=4)
wkago = today - timedelta(days=6)
today, wkago

(datetime.date(2021, 12, 30), datetime.date(2021, 12, 24))

### Restart & Run All Cells

### This process affects only already in port stocks. To highlight price changes in percent.

In [3]:
sql = '''
SELECT * FROM buy ORDER BY date DESC LIMIT 1
'''
df_buy = pd.read_sql(sql, const)
df_buy

,name,date,volbuy,price,volsell,volbal,active,dividend,period,grade
0,IMH,2021-12-08,"6,000.00",11.00,0.00,0.00,1,None,3,A3


In [4]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks 


SELECT * FROM stocks WHERE name = 'IMH'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,729,IMH,mai,12.60,16.20,2.08,9.28,3.80,107.50,2709,34.04,0.36,730,2021-08-22 15:13:27.650624,2021-12-31 04:05:04.853082


In [5]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 1
'''
sql = sql % name
df_price = pd.read_sql(sql, const)
df_price

,name,date,price,maxp,minp,qty,opnp
0,IMH,2021-12-30,12.60,13.10,12.50,5340157,12.50


In [6]:
col9 = 'name price_d price_w pct perform mrkt '.split()

In [7]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (today, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2021-12-30' AND name IN ('BCH', 'BGRIM', 'CKP', 'DCC', 'DIF', 'DOHOME', 'EPG', 'GLOBAL', 'HREIT', 'IMH', 'IP', 'IVL', 'JASIF', 'KBANK', 'KCE', 'MCS', 'NER', 'NOBLE', 'PTT', 'PTTGC', 'RATCH', 'RCL', 'RJH', 'SAT', 'SCC', 'SIS', 'STA', 'TISCO', 'TMT', 'TOP', 'TSTH', 'TU', 'TYCN')
ORDER BY name, date


33

In [8]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (wkago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2021-12-24' AND name IN ('BCH', 'BGRIM', 'CKP', 'DCC', 'DIF', 'DOHOME', 'EPG', 'GLOBAL', 'HREIT', 'IMH', 'IP', 'IVL', 'JASIF', 'KBANK', 'KCE', 'MCS', 'NER', 'NOBLE', 'PTT', 'PTTGC', 'RATCH', 'RCL', 'RJH', 'SAT', 'SCC', 'SIS', 'STA', 'TISCO', 'TMT', 'TOP', 'TSTH', 'TU', 'TYCN') 
ORDER BY name


33

In [9]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(33, 3)

In [10]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [11]:
trend['pct'] = (trend.price_d-trend.price_w)/trend.price_w * 100
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)
trend.sort_values(by=['pct'],ascending=[True])

,name,price_d,price_w,pct,perform
14,KCE,88.00,90.00,-2.22,Worse
25,SIS,44.00,44.75,-1.68,Worse
16,NER,7.00,7.10,-1.41,Worse
5,DOHOME,23.30,23.60,-1.27,Worse
4,DIF,13.90,14.00,-0.71,Worse
32,TYCN,3.58,3.60,-0.56,Worse
10,IP,19.50,19.60,-0.51,Worse
21,RCL,49.25,49.50,-0.51,Worse
7,GLOBAL,20.00,20.10,-0.50,Worse
8,HREIT,9.05,9.05,0.00,No Change


In [12]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,42.42%
No Change,30.30%
Worse,27.27%


In [13]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(358, 8)

In [14]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [15]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['pct'],ascending=[True]).shape

(33, 13)

In [16]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[col9].sort_values(by=['pct','name'],ascending=[True,True])

,name,price_d,price_w,pct,perform,mrkt
14,KCE,88.00,90.00,-2.22,Worse,SET50
7,GLOBAL,20.00,20.10,-0.50,Worse,SET50
18,PTT,38.00,38.00,0.00,No Change,SET50
20,RATCH,45.00,45.00,0.00,No Change,SET50
31,TU,19.50,19.50,0.00,No Change,SET50
19,PTTGC,58.75,58.50,0.43,Better,SET50
11,IVL,43.25,43.00,0.58,Better,SET50
29,TOP,49.50,49.00,1.02,Better,SET50
1,BGRIM,40.50,40.00,1.25,Better,SET50
24,SCC,386.00,380.00,1.58,Better,SET50


In [17]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,61.54%
No Change,23.08%
Worse,15.38%


In [18]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[col9].sort_values(by=['pct','name'],ascending=[True,True])

,name,price_d,price_w,pct,perform,mrkt
5,DOHOME,23.30,23.60,-1.27,Worse,SET100
2,CKP,5.00,5.00,0.00,No Change,SET100
0,BCH,20.10,19.90,1.01,Better,SET100


In [19]:
flt_set100[col9].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,33.33%
No Change,33.33%
Worse,33.33%


In [20]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[col9].sort_values(by=['pct','name'],ascending=[True,True])

,name,price_d,price_w,pct,perform,mrkt
25,SIS,44.00,44.75,-1.68,Worse,SET999
16,NER,7.00,7.10,-1.41,Worse,SET999
4,DIF,13.90,14.00,-0.71,Worse,SET999
32,TYCN,3.58,3.60,-0.56,Worse,SET999
21,RCL,49.25,49.50,-0.51,Worse,SET999
3,DCC,2.84,2.84,0.00,No Change,SET999
8,HREIT,9.05,9.05,0.00,No Change,SET999
17,NOBLE,6.15,6.15,0.00,No Change,SET999
22,RJH,32.75,32.75,0.00,No Change,SET999
28,TMT,10.30,10.30,0.00,No Change,SET999


In [21]:
flt_set999[col9].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
No Change,40.00%
Worse,33.33%
Better,26.67%


In [22]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[col9].sort_values(by=['pct','name'],ascending=[True,True])

,name,price_d,price_w,pct,perform,mrkt
10,IP,19.50,19.60,-0.51,Worse,mai
9,IMH,12.60,11.40,10.53,Better,mai


In [23]:
flt_setmai[col9].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,50.00%
Worse,50.00%
